In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import time

import plotly
import plotly.graph_objs as go
import chart_studio.plotly as py
from chart_studio.plotly import plot, iplot
import plotly.express as px

# Setting pandas to display columns
pd.set_option('display.max_columns', None)

In [13]:
nfl_small2_end_of_drive = pd.read_csv('nfl_small_end_of_drive.csv',index_col=1,\
    dtype= {'ARI' : 'str','ATL' : 'str', 'BAL' : 'str', 'BUF' : 'str', 'CAR' : 'str',
             'CHI' : 'str', 'CIN' : 'str', 'CLE' : 'str', 'DAL' : 'str', 'DEN' : 'str',
             'DET' : 'str', 'GB' : 'str', 'HOU' : 'str', 'IND' : 'str', 'JAX' : 'str',
             'KC' : 'str', 'LA' : 'str', 'LAC' : 'str', 'MIA' : 'str', 'MIN' : 'str',
             'NE' : 'str', 'NO' : 'str', 'NYG' : 'str','NYJ' : 'str', 'OAK' : 'str',
             'PHI' : 'str', 'PIT' : 'str','SEA' : 'str', 'SF' : 'str', 'TB' : 'str',
             'TEN' : 'str', 'WAS' : 'str'})


In [14]:
nfl_small2_end_of_drive.head()

,index,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,year,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Unnamed: 0_y,Wind Direction (deg),Wind Speed (km/h),added_time,game_name,end_of_drive,end_of_game,INT,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,week
play_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36,270409,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,NE,35.0,1,0,NaN,0.0,NE 35,0,0,S.Gostkowski kicks 65 yards from NE 35 to end ...,kickoff,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
51,270410,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,80.0,1,0,1.0,0.0,PIT 20,10,18,(15:00) De.Williams right tackle to PIT 38 for...,run,18.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
72,270411,2015091000,2015-09-10 00:00:00,14:21,861.0,1761.0,3561.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,62.0,1,0,1.0,0.0,PIT 38,10,31,(14:21) B.Roethlisberger pass short right to A...,pass,9.0,short,-4.0,13.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015-09-10 20:40:00,2015.0,0 days 00:01:57.000000000,2015-09-10 20:41:57,2015-09-10 20:41:57,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
101,270412,2015091000,2015-09-10 00:00:00,14:04,844.0,1744.0,3544.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,53.0,1,0,2.0,0.0,PIT 47,1,31,(14:04) De.Williams right guard to NE 49 for 4...,run,4.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:02:48.000000000,2015-09-10 20:42:48,2015-09-10 20:42:48,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
122,270413,2015091000,2015-09-10 00:00:00,13:26,806.0,1706.0,3506.0,Half1,0,1,NE,PIT,PIT,away,NE,NE,49.0,1,0,1.0,0.0,NE 49,10,45,(13:26) B.Roethlisberger pass short right to H...,pass,14.0,short,9.0,5.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015-09-10 20:40:00,2015.0,0 days 00:04:42.000000000,2015-09-10 20:44:42,2015-09-10 20:44:42,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-0

In [15]:
# Getting teams to add team matrix for easier filtering
teams = list(nfl_small2_end_of_drive.groupby('posteam').sum().index)
teams[:4]

['ARI', 'ATL', 'BAL', 'BUF']

In [16]:
nfl_small2_end_of_drive.year.unique()

array([2015., 2016., 2017., 2018.])

In [19]:
# Creating Home and Away Win Columns
nfl_small2_end_of_drive['home_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              > nfl_small2_end_of_drive['total_away_score'],1,0)

nfl_small2_end_of_drive['away_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              < nfl_small2_end_of_drive['total_away_score'],1,0)

In [23]:
sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2015) &\
                                            (nfl_small2_end_of_drive['CLE'] == 'H')
                                           ]['home_team_win'])

2

In [182]:
# Team wins overall for sorting
team_wins_overall = []
years = [2015,2016,2017,2018]
year_list = []
team_list = []

for team in teams:
    for year in years:
        home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'H')
                                               ]['home_team_win'])

        away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'A')
                                               ]['away_team_win'])

        # Due to lack of data, normalizing 2018 to 16 week season
        if year == 2018:
            all_wins = np.round((home_wins + away_wins) * 16/14,0)
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)
            
        else:
            all_wins = home_wins + away_wins
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)

team_by_wins = pd.DataFrame({'team':team_list,'wins':team_wins_overall,'year':year_list}).sort_values(by=['wins'])

In [212]:
team_win_list = list(team_by_wins.groupby('team').agg({'wins':'sum'}).sort_values(by=['wins']).index)

In [213]:
# Team wins overall for sorting
team_wins_overall = []
years = [2015,2016,2017,2018]
year_list = []
team_list = []

for team in team_win_list:
    for year in years:
        home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'H')
                                               ]['home_team_win'])

        away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'A')
                                               ]['away_team_win'])

        # Due to lack of data, normalizing 2018 to 16 week season
        if year == 2018:
            all_wins = np.round((home_wins + away_wins) * 16/14,0)
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)
            
        else:
            all_wins = home_wins + away_wins
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)

team_by_wins = pd.DataFrame({'team':team_list,'wins':team_wins_overall,'year':year_list})

In [214]:
# Could change this to mathematical bins
# bins = [0, 7, 10, 17]
labels = ['5 or Less', 'Between 6 and 9','10+ Wins']
team_by_wins['binned'] = pd.cut(team_by_wins['wins'], bins=3, labels=labels)

team_by_wins

,team,wins,year,binned
0,CLE,3.0,2015,5 or Less
1,CLE,1.0,2016,5 or Less
2,CLE,0.0,2017,5 or Less
3,CLE,7.0,2018,Between 6 and 9
4,SF,5.0,2015,5 or Less
...,...,...,...,...
123,KC,13.0,2018,10+ Wins
124,NE,12.0,2015,10+ Wins
125,NE,14.0,2016,10+ Wins
126,NE,13.0,2017,10+ Wins


In [215]:
team_by_wins.wins.describe() #.plot(kind='hist',bins=3)

count    128.000000
mean       7.960938
std        3.038862
min        0.000000
25%        6.000000
50%        8.000000
75%       10.000000
max       15.000000
Name: wins, dtype: float64

In [216]:
fig = go.Figure(data=[go.Bar(name='Total', x = team_by_wins['team'], y = team_by_wins['wins'])])

layout = go.Layout(
    title = 'Total Wins By Team (2015 - 2018)',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Wins',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'<b>'+'Total Wins by Team'+'</b>'+'<br>(2015 thru 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig

For the four years from 2015 thru 2018, we can naturally break teams into groups of successfulness, start with our top 7. Then we have quite a few middle of the pack teams, and an obvious group of bottom dwellers. One of our team member's favorite being the Cleveland Browns...

In [224]:
# Team wins by year
team_wins_2015 = []
teams = list(team_by_wins['team'].unique())
for team in teams:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2015) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2015) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = home_wins + away_wins
    
    team_wins_2015.append(round(all_wins,0))


team_wins_2016 = []
for team in teams:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2016) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2016) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = home_wins + away_wins
    
    team_wins_2016.append(round(all_wins,0))

team_wins_2017 = []
for team in teams:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2017) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2017) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = home_wins + away_wins
    
    team_wins_2017.append(round(all_wins,0))
    
team_wins_2018 = []
for team in teams:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2018) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2018) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = (home_wins + away_wins)*16/14
    
    team_wins_2018.append(round(all_wins,0))

In [227]:
fig = go.Figure(data=[go.Bar(name='2015', x = teams, y = team_wins_2015),\
                      go.Bar(name='2016', x = teams, y = team_wins_2016),\
                      go.Bar(name='2017', x = teams, y = team_wins_2017),\
                      go.Bar(name='2018', x = teams, y = team_wins_2018)])

layout = go.Layout(
    title = 'Total Wins By Year (2015 - 2018)',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Wins',ticklen= 5,zeroline= False)
    
)

fig.update_layout(
    title={
        'text':'<b>'+'Total Wins by Year'+'</b>'+'<br>(2015 thru 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        barmode='group',width=1000, height=600)
fig

We see mostly steady win totals, but not some teams have over or underpreformed. We are assuming Total Wins over a four year period is indicative of how good a team is.

-Needs to be corrected for proper win totals?   
-Needs dividers between teams?

In [157]:
team_by_wins.head()

,team,wins,year,binned
30,CLE,0,2017,5 or Less
29,CLE,1,2016,5 or Less
113,SF,2,2016,5 or Less
57,JAX,3,2016,5 or Less
3,ARI,3,2018,5 or Less


In [228]:
x2015 = team_by_wins[team_by_wins['year']==2015].groupby('binned').count().index
y2015 = team_by_wins[team_by_wins['year']==2015].groupby('binned').count()['team'].values

x2016 = team_by_wins[team_by_wins['year']==2016].groupby('binned').count().index
y2016 = team_by_wins[team_by_wins['year']==2016].groupby('binned').count()['team'].values

x2017 = team_by_wins[team_by_wins['year']==2017].groupby('binned').count().index
y2017 = team_by_wins[team_by_wins['year']==2017].groupby('binned').count()['team'].values

x2018 = team_by_wins[team_by_wins['year']==2018].groupby('binned').count().index
y2018 = team_by_wins[team_by_wins['year']==2018].groupby('binned').count()['team'].values

In [229]:
fig = go.Figure(data=[go.Bar(name='2015', x = x2015, y = y2015),\
                      go.Bar(name='2016', x = x2016, y = y2016),\
                      go.Bar(name='2017', x = x2017, y = y2017),\
                      go.Bar(name='2018', x = x2018, y = y2018)])

layout = go.Layout(
    title = 'Total Wins By Year (2015 - 2018)',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Wins',ticklen= 5,zeroline= False)
    
)

fig.update_layout(
    title={
        'text':'<b>'+'Total Wins by Year'+'</b>'+'<br>(2015 thru 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        barmode='group',width=1000, height=600)
fig

<h2>Creating end of game summary metrics</h2>

In [199]:
nfl_end_of_game = pd.read_csv('nfl_end_of_game.csv',index_col=0)
nfl_end_of_game.head()

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,win,pass_run_ratio,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,year,binned
0,2015091000,NE,PIT,28,21,366.0,429.0,23.0,25.0,34.0,42.0,1.0,2.0,0.0,0.0,4.0,4.0,281.0,332.0,82.0,134.0,1.0,1.48,-63.0,1.0,0.5,1.0,8.264706,3.565217,7.904762,5.360000,2015,10+ Wins
1,2015091300,CHI,GB,23,31,416.0,336.0,33.0,28.0,36.0,23.0,1.0,0.0,3.0,3.0,1.0,1.0,213.0,189.0,189.0,134.0,0.0,1.09,80.0,-1.0,1.0,1.0,5.916667,5.727273,8.217391,4.785714,2015,Between 6 and 9
2,2015091301,LA,SEA,33,29,354.0,338.0,26.0,32.0,29.0,47.0,3.0,1.0,2.0,2.0,3.0,3.0,276.0,219.0,76.0,127.0,1.0,1.12,16.0,-2.0,1.0,1.0,9.517241,2.923077,4.659574,3.968750,2015,Between 6 and 9
3,2015091302,JAX,CAR,9,19,255.0,253.0,21.0,33.0,44.0,33.0,4.0,1.0,2.0,2.0,4.0,4.0,169.0,158.0,96.0,107.0,0.0,2.10,2.0,-3.0,1.0,1.0,3.840909,4.571429,4.787879,3.242424,2015,5 or Less
4,2015091303,WAS,MIA,10,16,290.0,264.0,36.0,17.0,32.0,37.0,4.0,2.0,2.0,2.0,1.0,1.0,188.0,182.0,162.0,75.0,0.0,0.89,26.0,-2.0,1.0,1.0,5.875000,4.500000,4.918919,4.411765,2015,Between 6 and 9


In [200]:
nfl_end_of_game.columns

Index(['game_id', 'Team', 'team_against', 'score', 'score_against', 'yards',
       'yards_against', 'run_plays', 'run_plays_against', 'pass_plays',
       'pass_plays_against', 'to_for', 'to_against', 'fg_at', 'fg_md',
       'fg_at_against', 'fg_md_against', 'pass_yards', 'pass_yards_against',
       'run_yards', 'run_yards_against', 'win', 'pass_run_ratio', 'yard_diff',
       'to_diff', 'fg_rate', 'fg_rate_against', 'pass_yds_per_at',
       'run_yds_per_at', 'pass_yds_per_at_against', 'run_yds_per_at_against',
       'year', 'binned'],
      dtype='object')

In [201]:
# Adding numeric win group to allow for correlation
win_dict = {'36+ Wins':2,'35 or Less':1,'24 or Less':0}
nfl_end_of_game['win_group_num'] = nfl_end_of_game['binned'].map(win_dict)

In [202]:
corr_cols = ['yards','yards_against', 'run_plays', 'run_plays_against', 'pass_plays',\
       'pass_plays_against','yard_diff','to_for', 'to_against', 'to_diff','fg_rate',\
        'fg_rate_against','pass_yds_per_at','pass_run_ratio','run_yds_per_at', 
             'pass_yds_per_at_against', 'run_yds_per_at_against']

In [203]:
corr_df = nfl_end_of_game[corr_cols].apply(lambda x: x.corr(nfl_end_of_game.win)).sort_values(kind="quicksort")

In [204]:
fig = go.Figure(data=[go.Bar(name='Corr', x = corr_df.index, y = corr_df.values)])

layout = go.Layout(
    title = 'Variables by Win Correlation',
    xaxis= dict(title= 'Variable',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Correlation',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'<b>'+'Variables by Win Correlation'+'</b>'+'<br>(Based on Individual Game Results)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig

As you can imagine, factors such as Total Yards and Yard Differential play a factor in wins, but so does turnovers (TOs), Pass to Run Ratio, and Pass Yards Per Attempt, which may change as weather changes

<h2>Keying in on some individual variables</h2>

In [233]:
color_dict = {'10+ Wins': 'rgb(215,48,39)', 'Between 6 and 9': 'rgb(253,253,32)','5 or Less': 'rgb(42,55,230)'}
color_group = nfl_end_of_game['binned'].map(color_dict)

In [236]:
nfl_end_of_game.head()

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,win,pass_run_ratio,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,year,binned,win_group_num
0,2015091000,NE,PIT,28,21,366.0,429.0,23.0,25.0,34.0,42.0,1.0,2.0,0.0,0.0,4.0,4.0,281.0,332.0,82.0,134.0,1.0,1.48,-63.0,1.0,0.5,1.0,8.264706,3.565217,7.904762,5.360000,2015,10+ Wins,NaN
1,2015091300,CHI,GB,23,31,416.0,336.0,33.0,28.0,36.0,23.0,1.0,0.0,3.0,3.0,1.0,1.0,213.0,189.0,189.0,134.0,0.0,1.09,80.0,-1.0,1.0,1.0,5.916667,5.727273,8.217391,4.785714,2015,Between 6 and 9,NaN
2,2015091301,LA,SEA,33,29,354.0,338.0,26.0,32.0,29.0,47.0,3.0,1.0,2.0,2.0,3.0,3.0,276.0,219.0,76.0,127.0,1.0,1.12,16.0,-2.0,1.0,1.0,9.517241,2.923077,4.659574,3.968750,2015,Between 6 and 9,NaN
3,2015091302,JAX,CAR,9,19,255.0,253.0,21.0,33.0,44.0,33.0,4.0,1.0,2.0,2.0,4.0,4.0,169.0,158.0,96.0,107.0,0.0,2.10,2.0,-3.0,1.0,1.0,3.840909,4.571429,4.787879,3.242424,2015,5 or Less,NaN
4,2015091303,WAS,MIA,10,16,290.0,264.0,36.0,17.0,32.0,37.0,4.0,2.0,2.0,2.0,1.0,1.0,188.0,182.0,162.0,75.0,0.0,0.89,26.0,-2.0,1.0,1.0,5.875000,4.500000,4.918919,4.411765,2015,Between 6 and 9,NaN


In [257]:
nfl_end_of_game.groupby('binned').agg({'pass_run_ratio':'mean'}).sort_values(by=['binned'])

,pass_run_ratio
binned,
10+ Wins,1.468787
5 or Less,1.708238
Between 6 and 9,1.662533


In [258]:
# Create categorical variable and order for binned
nfl_end_of_game['binned'] = pd.Categorical(nfl_end_of_game['binned'], \
                                        categories=['5 or Less','Between 6 and 9','10+ Wins'],\
                                        ordered=True)

In [265]:
fig = go.Figure()

fig.add_trace(go.Bar(name='pass to run',x=nfl_end_of_game.groupby('binned').agg({'pass_run_ratio':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'pass_run_ratio':'mean'})['pass_run_ratio'])
             )

fig.update_layout(
    title={
        'text':'<b>'+'Pass to Run Ratio'+'</b>'+'<br>(Pass Plays / Run Plays)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig

In [268]:
fig = go.Figure()

fig.add_trace(go.Bar(name='TO Margin',x=nfl_end_of_game.groupby('binned').agg({'to_diff':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'to_diff':'mean'})['to_diff'])
             )

fig.update_layout(
    title={
        'text':'<b>'+'Turnover Margin'+'</b>'+'<br>(Giveaways Minus Takeways)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig

In [269]:
nfl_end_of_game.head(2)

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,win,pass_run_ratio,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,year,binned,win_group_num
0,2015091000,NE,PIT,28,21,366.0,429.0,23.0,25.0,34.0,42.0,1.0,2.0,0.0,0.0,4.0,4.0,281.0,332.0,82.0,134.0,1.0,1.48,-63.0,1.0,0.5,1.0,8.264706,3.565217,7.904762,5.360000,2015,10+ Wins,NaN
1,2015091300,CHI,GB,23,31,416.0,336.0,33.0,28.0,36.0,23.0,1.0,0.0,3.0,3.0,1.0,1.0,213.0,189.0,189.0,134.0,0.0,1.09,80.0,-1.0,1.0,1.0,5.916667,5.727273,8.217391,4.785714,2015,Between 6 and 9,NaN


In [291]:
fig = go.Figure()

fig.add_trace(go.Bar(name='TO Margin',x=nfl_end_of_game.groupby('binned').agg({'fg_rate':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'fg_rate':'mean'})['fg_rate'])
             )

fig.update_layout(
    title={
        'text':'<b>'+'Field Goal Success Rate'+'</b>'+'<br>(Field Goals Made versus Attempted)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig

In [327]:
# Yard Differential
fig = go.Figure()

fig.add_trace(go.Bar(name='Yard Diff',x=nfl_end_of_game.groupby('binned').agg({'yard_diff':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'yard_diff':'mean'})['yard_diff'])
             )

fig.update_layout(
    title={
        'text':'<b>'+'Game Average Yard Differential'+'</b>'+'<br>(Yards Gained Minus Yard Given Up)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.update_xaxes(range=[-.5, 2.5])
fig.update_yaxes(range=[-40, 40])

fig.add_shape(
            type="line",
            xref="paper",
            yref="paper",
            x0=0.033,
            y0=.1,
            x1=.3,
            y1=.1,
            line=dict(
                color="DarkOrange",
                width=3,
            ),
        )

fig

In [329]:
nfl_end_of_game.head(2)

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,win,pass_run_ratio,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,year,binned,win_group_num
0,2015091000,NE,PIT,28,21,366.0,429.0,23.0,25.0,34.0,42.0,1.0,2.0,0.0,0.0,4.0,4.0,281.0,332.0,82.0,134.0,1.0,1.48,-63.0,1.0,0.5,1.0,8.264706,3.565217,7.904762,5.360000,2015,10+ Wins,NaN
1,2015091300,CHI,GB,23,31,416.0,336.0,33.0,28.0,36.0,23.0,1.0,0.0,3.0,3.0,1.0,1.0,213.0,189.0,189.0,134.0,0.0,1.09,80.0,-1.0,1.0,1.0,5.916667,5.727273,8.217391,4.785714,2015,Between 6 and 9,NaN


In [342]:
# Yard Differential
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Yard Diff',x=nfl_end_of_game.groupby('binned').agg({'yard_diff':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'yard_diff':'mean'})['yard_diff']),
                    secondary_y=False
             )

fig.add_trace(go.Scatter(name='Yards Gained',x=nfl_end_of_game.groupby('binned').agg({'yards':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'yards':'mean'})['yards'],yaxis='y2'),
                        secondary_y=True
             )

fig.add_trace(go.Scatter(name='Yards Given Up',x=nfl_end_of_game.groupby('binned').agg({'yards_against':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'yards_against':'mean'})['yards_against']*-1,yaxis='y2'),
                        secondary_y=True
             )

fig.update_layout(
    title={
        'text':'<b>'+'Game Average Yard Differential'+'</b>'+'<br>(Yards Gained Minus Yard Given Up)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
    
    )

fig

<h2>Graveyard</h2>

In [74]:
fig = go.Figure(data=[go.Bar(name='Avg Wins', x = team_by_wins['team'], y = team_by_wins['wins']/4),\
                     go.Bar(name='Avg Wins', x = team_by_wins['team'], y = np.repeat(.1,32))])

layout = go.Layout(
    title = 'Average Wins By Team (2015 - 2018)',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Wins',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'<b>'+'Total Wins by Team'+'</b>'+'<br>(2015 thru 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},barmode='stack',
        shapes=[{'type': 'line','y0':7,'y1': 7,'x0':'CLE', 
                              'x1':'CLE','xref':'x1','yref':'y1',
                              'line': {'color': 'red','width': 2.5}}])
fig

I want this to be average wins over four years, with year totals plotted as lines